# Dataset Merge IDEA

1. history에 대해 drop duplicate 후 drop column      
2. watch 에 대해서도 drop duplicate 후 drop column     
3. 이후 inner join       
4. 다른 dataset merge   

문제 : 3번부터 램이 터져나감    
대안 1. 나눠서 조인 (노가다...)   
대안 2. 더 나은 노트북 환경 찾기   
대안 3. 똑똑하게 머리 쓰기 **<해결 방안 찾아냄>**

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

### 시청 시작 데이터에서 row 중복 체크 - 1만 6천개 가량의 중복 확인. Drop

In [2]:
'''
의심의 여지 없이 중복 존재 
아래 코드 실행 시 groupby로 상당히 많이 묶이고, 최종 column 개수가 899021로 줄었음을 알 수 있음
''' 
# history.groupby(['profile_id','log_time','ss_id']).count().sort_values('album_id',ascending=False)[:60]

'\n의심의 여지 없이 중복 존재 \n아래 코드 실행 시 groupby로 상당히 많이 묶이고, 최종 column 개수가 899021로 줄었음을 알 수 있음\n'

In [3]:
history = pd.read_csv('../input/lgground/history_data.csv')
history.drop_duplicates(subset=['profile_id','log_time','album_id'],inplace=True) 
# row = 1005651 rows × 8 columns
# after drop duplicates 899021 rows × 8 columns
# 106630 개 row 중복

In [4]:
# 'continuous_play','short_trailer','log_time' -- drop
# payment 결측 0으로 대체
# history.drop(labels=['log_time','act_target_dtl','continuous_play','short_trailer'],axis=1).fillna(0,inplace=True)

### Watch에 대해서도 동일한 작업 수행 (duplicate drop, feature selection)

In [5]:
watch = pd.read_csv('../input/lgground/watch_e_data.csv')
watch.drop_duplicates(subset=['profile_id','log_time','album_id'],inplace=True) 
# watch.drop(labels=['log_time','act_target_dtl'],axis=1,inplace=True)  # column 드랍. 따로 처리할 결측치 없음

In [6]:
# full 병합 코드는 폭탄 그 자체
# 돌렸다 하면 터져용 
# datamerge = pd.merge(history,watch,how='inner',on='profile_id')
# datamerge.to_csv('../input/lgground')

## 터진다...! 💣💥
*근데 여기서 문득 드는 생각*

1. 반드시 history와 watch를 inner join으로 묶어야 할까? 그러니까... serve할 비중복 유저 리스트만 history에서 가져오고, 제대로 된 neg/pos sampling이 가능한 watch 데이터에 붙여서 그것 중심으로 학습할 수도 있지 않나?     
애초에 history에서 유의미한 column은 **시청 시작 데이터에만 있는 profile_id & 그나마 payment 뿐이다**    
ㄴ *그나마 payment* : payment는 영상 시청의 장벽 같은 요소로 작동하기 때문에 (무료 영상이 유료 영상보다 많이 재생된다) sampling 혹은 추천 요소에 중요하게 고려할 만 하다.    
.       
           
이렇게 접근하면 어떤 방식이 가능해지냐면 < log_time column도 날릴 수 있게 된다 :: 그래도 train-vaild를 위해서 날리진 말자 >    
1. 두 dataset에서 필요 없는 column을 drop하고       
2. history에만 있는 profile_id 가 있는 row를 전부 추출해 watch에 append한다.
3. 추후 데이터셋 분할을 고려해, log_time 기준으로 dataframe 정렬 sort.    
3. 위 데이터셋에 meta + profile 데이터셋 merge 수행한다.    
     
> 최종적으로는 watch에 있는 중요한 값을 기본으로 가지고, serve대상인 유저목록까지 챙긴 dataset이 됨     

In [7]:
# data preprocessing
# history에만 있는 profile_id  Length: 8311 / watch에만 있는 profile_id  Length: 7658
m_his = history['profile_id'].drop_duplicates() # 터짐 방지로 profile_id column만 남김 
m_wat = watch['profile_id'].drop_duplicates()
id_only_in_history = pd.merge(m_his,m_wat,how='outer',indicator=True
                ).query('_merge == "left_only"').drop(columns=['_merge'])
id_only_in_history_list = id_only_in_history['profile_id'].to_list()
id_only_in_history_rows = history[history['profile_id'].isin(id_only_in_history_list)]
id_only_in_history_rows = id_only_in_history_rows.drop_duplicates(subset=['profile_id','ss_id','log_time','album_id']) # 15241 rows
id_only_in_history_rows.drop(columns=['continuous_play','short_trailer'])
dataset = watch.append(id_only_in_history_rows,sort=False).sort_values('log_time').reset_index(drop=True)

In [8]:
# dataset verification
# watch : 800632 +15241 = row 815873 나와야 정상병합
print(dataset['total_time'].isnull().sum()) #제대로 합쳐졌다면 결측이 15241개 있을 것
print(dataset['profile_id'].nunique()) #제대로 합쳐졌다면 8311명일 것 

15241
8311


In [9]:
dataset.to_csv('./dataset.csv') 
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 815873 entries, 0 to 815872
Data columns (total 10 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   profile_id       815873 non-null  int64  
 1   ss_id            815873 non-null  int64  
 2   log_time         815873 non-null  int64  
 3   act_target_dtl   815873 non-null  object 
 4   album_id         815873 non-null  int64  
 5   watch_time       800632 non-null  float64
 6   total_time       800632 non-null  float64
 7   continuous_play  815873 non-null  object 
 8   payment          564 non-null     float64
 9   short_trailer    15241 non-null   object 
dtypes: float64(3), int64(4), object(3)
memory usage: 62.2+ MB


# Meta Data merge 

In [10]:
# # 메타데이터 병합 
meta = pd.read_csv('../input/lgground/meta_data.csv') # 42602 rows
meta_p = pd.read_csv('../input/lgground/meta_data_plus.csv') # 767948 rows
# meta_merge = pd.merge(meta,meta_p,how='inner',on='album_id') #### meta 정보 최종 집합체 : 832356 rows
# data_w_meta = pd.merge(dataset,meta,how='inner',on='album_id')

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3552: DtypeWarning: Columns (15) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


# 문제발생.... 😵

meta 와 meta plus 데이터의 총 row 개수가 많이 차이난다. 그냥 merge해서 해결될 문제는 아님   
복잡하구마안   
  
> 아 맞다... meta_plus는 앨범 하나에 여러 개의 키워드가 할당되어 있었지...    
preprocessing 해야 한다!     
추후 모델 학습에 지장이 없을 형태로 바꿔두려면.... 흠....
>> 유니크한 album 개수는 똑같다
    
일단 meta_plus['keyword_name'].nunique()는 1062개. 이거에 대한 인코딩을 진행해서 다시 붙여주면 되지 않을까?     
+ keyword_type의 nunique도 1062개일까? (나름 인코딩이 된 걸까?) >> 아니다! 1101개다! (ㅎㅎ.. 뭐야 왜있는거람)  
++ 그나마 다행? : [keyword_name]에 결측 row는 없다. 인코딩만 잘하면 됨 

keyword value도 엄청 신경쓰인다... 제대로 분별력을 가진 데이터라고 믿어보는수밖에 ㅠ.ㅠ     
일단은 keyword_name에 feature encoding을 진행한다. 이후 keyword_type은 drop     

## Meta_plus 데이터 proprocess 

In [11]:
meta_p['keyword_type'].nunique()

1101

In [12]:
# display(meta_p[meta_p['album_id']==390]) 

# 1개 키워드 개수는 30개 가량뿐이다. 일단 keyword를 신뢰할 만하다고 판단.
# 5세와 5세 추천은 합치지 않는 게 낫다. 사유는 아래 코드 실행 
# 다른 나이를 위해 만든 영상임에도 5세가 볼만하다면 제시되는 키워드. 5세와 5세 추천 키워드가 같이 쓰인 앨범은 없다는 게 조금 신경쓰인다.
# display(meta_p[(meta_p['keyword_name']=='5세 추천')&(meta_p['keyword_name']=='5세')]) 
# 중복 키워드 결과 없음
# display(meta_p[(meta_p['keyword_name']=='비교')&(meta_p['keyword_name']=='비교하기')])  
# display(meta_p[(meta_p['keyword_name']=='야외놀이')&(meta_p['keyword_name']=='야외놀이안전')]) 
# display(meta_p[(meta_p['keyword_name']=='언어1')&(meta_p['keyword_name']=='언어2')]) 
# display(meta_p[(meta_p['keyword_name']=='언어지능1')&(meta_p['keyword_name']=='언어지능2')]) 

In [13]:
pd.set_option('display.max_rows',None)
# meta_p['keyword_name'].value_counts() # 1개 키워드 개수는 30개 가량뿐이다. 일단 keyword를 신뢰할 만하다고 판단.

# 한편, 아주 비슷한 keyword끼리는 encoding 전에 같은 태그로 통일할 필요가 있음. (ex. 치아 구강 ↔ 치아,  용감함 ↔ 용감함4)
# 이걸 확인하기 위해서, sort로 1062개를 정렬해본다. (휴먼 러닝을 피할 수가 없음)

key_count = pd.DataFrame(meta_p['keyword_name'].value_counts().reset_index(drop=False).sort_values(by='index'))
# key_count

### 확인해 봐야 하는 (합쳐야 하는 단어 목록)
5세(05010403) > 5세 추천(050b0102)    
6세 > 6세 추천(050b0103)    
7세 > 7세 추천    
비교 < 비교하기    
안데르센 > 안데르센상    
야외놀이 > 야외놀이안전 (확인필요)    
언어1, 언어2 (확인필요)     
언어지능1, 언어지능2    
용감함 > 용감함 4    
의사소통 < 의사소통(Communication)    
자신감 < 자신감(Confidence)    
장선혜 > 장선혜(작화)    
최숙희 > 최숙희(작화)    
캐리 < 캐리TV
> 숫자가 더 적은 쪽을 변경하기로 한다. 

In [14]:
# 태그 코드는 떨어트릴 거지만 그래도 같이 변경해주자.

meta_p.replace({'keyword_name' : '안데르센상'}, '안데르센',inplace=True)
meta_p.replace({'keyword_name' : '용감함 4'}, '용감함',inplace=True)
meta_p.replace({'keyword_name' : '의사소통'}, '의사소통(Communication)',inplace=True)
meta_p.replace({'keyword_name' : '자신감'}, '자신감(Confidence)',inplace=True)
meta_p.replace({'keyword_name' : '장선혜'}, '장선혜(작화)',inplace=True)
meta_p.replace({'keyword_name' : '최숙희'}, '최숙희(작화)',inplace=True)
meta_p.replace({'keyword_name' : '캐리'}, '캐리TV',inplace=True)

meta_p.replace({'keyword_type' : '05010a01'}, '0501090d',inplace=True)
meta_p.replace({'keyword_type' : '0509071c'}, '05090708',inplace=True)
meta_p.replace({'keyword_type' : '05030908'}, '050c1002',inplace=True)
meta_p.replace({'keyword_type' : '05070607'}, '050c1006',inplace=True)
meta_p.replace({'keyword_type' : '05010907'}, '05010914',inplace=True)
meta_p.replace({'keyword_type' : '05010909'}, '05010915',inplace=True)
meta_p.replace({'keyword_type' : '05080409'}, '0501080a',inplace=True)

In [15]:
meta_p['keyword_type'].nunique()
#제대로 정리됨

1094

# (meta+plus)data Merge 작업 

In [16]:
# sub_title 리스트 담기
meta_df_sub_title_list_dict= {}

for idx, row in meta.iterrows():
    if meta_df_sub_title_list_dict.get(row['album_id'], None) == None:
        meta_df_sub_title_list_dict[row['album_id']] = [row['sub_title']]
    elif row['sub_title'] not in meta_df_sub_title_list_dict[row['album_id']]:
        meta_df_sub_title_list_dict[row['album_id']] = meta_df_sub_title_list_dict[row['album_id']]+[row['sub_title']]
        
print('전체 컨텐츠 개수:',len(set(meta.album_id)),'딕셔너리 담긴 아이템 개수:',len(meta_df_sub_title_list_dict))        

meta_df_prepro_sub_title = meta.copy()
meta_df_prepro_sub_title['sub_title'] = meta_df_prepro_sub_title['album_id'].apply(lambda x: meta_df_sub_title_list_dict.get(x, None))

전체 컨텐츠 개수: 39875 딕셔너리 담긴 아이템 개수: 39875


In [17]:
# 전체 중복 삭제하기
meta_df_prepro_sub_title = meta_df_prepro_sub_title.drop_duplicates(subset=['album_id'], ignore_index=True)

print('전체 데이터수와 album_id 개수 동일 확인:', len(meta_df_prepro_sub_title), len(meta_df_prepro_sub_title.album_id.unique()))

전체 데이터수와 album_id 개수 동일 확인: 39875 39875


In [18]:
# 메타 플러스 데이터 keyword(keyword_name, type)리스트 담기
meta_plus_type_name = meta_p.copy()

In [19]:
type_to_name = {}

for idx, row in meta_p.iterrows():
    type_to_name[row['keyword_type']] = row['keyword_name']
    
meta_plus_type_name['keyword_name'] = meta_plus_type_name['keyword_type'].apply(lambda x: type_to_name[x])    

name_to_type = {}
for idx, row in meta_p.iterrows():
    name_to_type[row['keyword_name']] = row['keyword_type']
    
meta_plus_type_name['keyword_type'] = meta_plus_type_name['keyword_name'].apply(lambda x: name_to_type[x])
print('keyword_name, keyword_type 개수 확인:',len(set(meta_plus_type_name.keyword_name)), len(set(meta_plus_type_name.keyword_type)))
print('앨범 당 키워드 중복 여부 확인:',len(meta_p),len(meta_p.drop_duplicates(subset=['album_id','keyword_type'])))

keyword_name, keyword_type 개수 확인: 1055 1055
앨범 당 키워드 중복 여부 확인: 767948 766943


In [20]:
meta_plus_df_prepro_keyword = meta_plus_type_name.copy()

In [21]:
# meta plus data type name 동일하게 만들기

meta_df_keyword_name_list_dict= {}
meta_df_keyword_type_list_dict= {}

for idx, row in meta_plus_df_prepro_keyword.iterrows():
    if meta_df_keyword_name_list_dict.get(row['album_id'], None) == None:
        meta_df_keyword_name_list_dict[row['album_id']] = [row['keyword_name']]
    elif row['keyword_name'] not in meta_df_keyword_name_list_dict[row['album_id']]:
        meta_df_keyword_name_list_dict[row['album_id']] = meta_df_keyword_name_list_dict[row['album_id']]+[row['keyword_name']]

for idx, row in meta_plus_df_prepro_keyword.iterrows():
    if meta_df_keyword_type_list_dict.get(row['album_id'], None) == None:
        meta_df_keyword_type_list_dict[row['album_id']] = [row['keyword_type']]
    elif row['keyword_type'] not in meta_df_keyword_type_list_dict[row['album_id']]:
        meta_df_keyword_type_list_dict[row['album_id']] = meta_df_keyword_type_list_dict[row['album_id']]+[row['keyword_type']]        

meta_plus_df_prepro_keyword['keyword_name'] = meta_plus_df_prepro_keyword['album_id'].apply(lambda x: meta_df_keyword_name_list_dict.get(x, None))
meta_plus_df_prepro_keyword['keyword_type'] = meta_plus_df_prepro_keyword['album_id'].apply(lambda x: meta_df_keyword_type_list_dict.get(x, None))

meta_plus_df_keyword = meta_plus_df_prepro_keyword.drop_duplicates(subset=['album_id'],ignore_index=True).drop(columns=['keyword_value'])

In [22]:
# meta data merge
all_meta_df = pd.merge(meta_df_prepro_sub_title, meta_plus_df_keyword, how='left',on='album_id')
all_meta_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 39875 entries, 0 to 39874
Data columns (total 18 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   album_id      39875 non-null  int64  
 1   title         39875 non-null  object 
 2   sub_title     39875 non-null  object 
 3   genre_large   39875 non-null  object 
 4   genre_mid     39875 non-null  object 
 5   genre_small   12279 non-null  object 
 6   country       31773 non-null  object 
 7   run_time      39875 non-null  int64  
 8   onair_date    5208 non-null   float64
 9   cast_1        26309 non-null  object 
 10  cast_2        21217 non-null  object 
 11  cast_3        15885 non-null  object 
 12  cast_4        11980 non-null  object 
 13  cast_5        6146 non-null   object 
 14  cast_6        2540 non-null   object 
 15  cast_7        743 non-null    object 
 16  keyword_type  39875 non-null  object 
 17  keyword_name  39875 non-null  object 
dtypes: float64(1), int64(2), o

In [23]:
all_meta_df.to_csv('./all_meta_df.csv')

# 최종! dataset(history+watch) + all_meta_df(meta+plus) 데이터 merge 
## payment 까지 반영 

In [28]:
data1 = pd.merge(dataset, all_meta_df, how='left',on='album_id')
# 아래처럼 병합하면 data2 의 column명에 _x, _y같은 값이 붙어있는 문제 발생 
# data2 = pd.merge(data1,history, how='left', on=['ss_id','album_id'])


# payment 정보 더하기  
'''
history에서 [payment/log_time/album_id]만 가진 dataframe 만들기
data1에 left join 하는데, 기준을 album_id, log_time 
== 같은 시점에 유료였던 것은 유료로 값이 들어가겠죠? (payment는 시간을 고려할수밖에 없습니다)
ㄴ (보류) 방금 진수님이 이야기한 걸 바탕으로는 : 유저 개개인도 payment를 따지는 데 중요한 고려 요소인 것 같아요? <추후 확인>
   유저가 한번 결제한 영상 > 
'''

history_pay = history[['payment','ss_id','album_id']].drop_duplicates()
data = pd.merge(data1, history_pay, how='left', on=['payment','ss_id','album_id'])

In [29]:
data.isnull().sum()
# log time, album on으로 merge : 815309
# ss_id, album on으로 merge :  815309

# 둘 다 똑같네 . . . 

profile_id              0
ss_id                   0
log_time                0
act_target_dtl          0
album_id                0
watch_time          15241
total_time          15241
continuous_play         0
payment            815309
short_trailer      800632
title                   0
sub_title               0
genre_large             0
genre_mid               0
genre_small        689896
country            188209
run_time                0
onair_date         799993
cast_1             322574
cast_2             418002
cast_3             481612
cast_4             548440
cast_5             699079
cast_6             793844
cast_7             807826
keyword_type            0
keyword_name            0
dtype: int64

In [26]:
data.to_csv('./data.csv')

In [27]:
# pay_his = history[['profile_id','album_id','ss_id']].drop_duplicates()
# dataset = pd.merge(dataset,pay_his,how='inner',on=['profile_id','album_id','ss_id'])

## 잊지 말아야 할 것
> history의 nunique한 아이템 수 : 대략 2만건   
mata의 nunique한 아이템 수 : 대략 4만건   
== 2달간의 소비데이터에 등장조차 하지 않은 아이템이 2만개나 있다

> **todo** 한 달씩 분할해서 / 첫 달에 등장한 아이템이 두번째 달에 많이 등장하는지 확인해보면 될 것 

# feature encoding


. . . loading

# Neg Samplig

시청하지 않은 것에 대해서 [시청 시간 feature]를 넣어줘야 함     
ㄴ 시청하지 않은 영상이 너무 많아서 네거티브 샘플링을 하면 편중/과적합이 된다     
ㄴ 관련 논문이나 연구가 있을 법 한데 (찾아보기)


진수's idea
어차피 랜덤하게 샘플링하니까 시청 시간도 랜덤하게 값을 넣어보자 (ex. 해당 아이템의 평균 시청시간)
*세션이 해당 유저가 들어온 시간이라면, 그 시간에 대해서는 로그*
유저가 연속해서 재생한 것과, 종료한 다음 재생한 것에 대해 기록에 차이를 줘야 하지 않나?